In [10]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.HRDaDek1yq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.RJTPXfPVja/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.lNvMwTQFRJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [11]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://deb.debian.org/debian buster InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://deb.debian.org/debian buster-updates InRelease
Hit:6 http://deb.debian.org/debian-security buster/updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (131 kB/s)
Reading package lists... Done
R

In [12]:
!pip install Selenium==4.9.0

In [69]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

# from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from selenium.common.exceptions import ElementClickInterceptedException

from time import sleep

import numpy as np

In [70]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox') # needed, because colab runs as root
    options.add_argument('--headless')  # or use pyvirtualdiplay
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

In [71]:
import glob
import os
from google.colab import files

def descarrega_xlsx(nou_nom):
    # Busca tots els arxius amb extensió .xlsx a la carpeta arrel de Colab
    xlsx_files = glob.glob('*.xlsx')
    print("Arxius trobats:", xlsx_files)

    if not xlsx_files:
        print("No s'han trobat arxius .xlsx")
        return

    # Ordena els arxius per data de modificació (del més recent al més antic)
    xlsx_files.sort(key=os.path.getmtime, reverse=True)

    # Agafa el primer arxiu (el més recent)
    ultim_arxiu = xlsx_files[0]
    print(f"Arxiu més recent trobat: {ultim_arxiu}")

    # Renombrar l'arxiu
    nou_nom_complet = nou_nom + ".xlsx"
    os.rename(ultim_arxiu, nou_nom_complet)
    print(f"Arxiu renombrat a: {nou_nom_complet}")

    # Descarregar l'arxiu amb el nou nom
    files.download(nou_nom_complet)


In [72]:
from urllib.parse import quote
import glob
from google.colab import files

# Busca tots els arxius amb extensió .xlsx abans de la descàrrega
xlsx_files_before = set(glob.glob('*.xlsx'))

def titular(codi):
  # URL del lloc web
  url = "https://branddb.wipo.int/en/advancedsearch/results?sort=score%20desc&strategy=concept&asStructure=%7B%22_id%22:%22aff0%22,%22boolean%22:%22AND%22,%22bricks%22:%5B%7B%22_id%22:%22aff1%22,%22key%22:%22applicant%22,%22strategy%22:%22Simple%22,%22value%22:%22%5C%22"+quote(codi)+"%5C%22%22%7D%5D%7D&_=1720017059217&fg=_void_"

  # Obre el lloc web
  driver.get(url)

  # Espera que el botó "Download results" sigui visible i clicable
  wait = WebDriverWait(driver, 20)
  download_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Download results"]')))
  download_button.click()

  # Espera que l'opció "EXCEL" sigui visible i clicable
  excel_option = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/app-root/div/div/page-results/results/div/search-results-menu/div/w-table-commands/ul/f-facet[4]/li/div/div[2]/span')))
  excel_option.click()

  # Dona temps perquè la descàrrega es completi
  time.sleep(10)  # Ajusta aquest temps segons sigui necessari

  descarrega_xlsx(codi)


In [74]:
titular("ajuntament de barcelona")

Arxius trobats: ['generalitat de catalunya.xlsx', 'nou_nom_del_arxiu.xlsx', 'GBD-advancedsearch_20240703.xlsx']
Arxiu més recent trobat: GBD-advancedsearch_20240703.xlsx
Arxiu renombrat a: ajuntament de barcelona.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>